# js预解析

In [3]:
%%html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <script>
        var otherBefore = 111;
    </script>
    <script>
        // 1问
        console.log(num);
        // 2问
        console.log('num=', num); //坑1： 此处并不会报错,而是输出num= undefined
        var num = 10;
        /*
         变量提升,相当于 
         var num;
         console.log('num=', num); 
         num = 10;
        */
        // 3问
        // function fn() {
        //     console.log('fn() exec...', 11);
        // }
        // fn(); //fn()能调用
        fn(); //fn()能调用吗？也是可以的
        function fn() {
            console.log('fn() exec...', 11);
        }
        // 4问
        // var fun = function() {
        //     console.log('fun() exec...', 22);
        // }
        // fun();//fun()能调用
        // fun();//fun()能调用吗? 不能，报错: Uncaught TypeError: fun is not a function
        var fun = function () {
            console.log('fun() exec...', 22);
        }
        /*
         fun是变量，变量提升,fun不是函数
         */
        // 1.js引擎运行js分为两步: 1.预解析 2.代码执行
        // (1).预解析：js引擎会将js里面所有的var和函数提升到当前作用域的最前面
        // (2).代码执行：按照代码的书写顺序执行
        // 2.预解析分为 变量预解析（变量提升） 和 函数预解析（函数提升)
        // (1).变量提升：就是把变量声明提升到当前作用域的最前面，不提升赋值（****）
        // (2).函数提升：就是把所有的使用函数关键字function声明的函数提升到当前作用域的最前面，fun=function(){}不会提升，这是函数表达式
        console.log('otherBefore=', otherBefore); //otherBefore=111
        console.log('otherAfter=', otherAfter); //报错: Uncaught ReferenceError: otherAfter is not defined
    </script>
    <script>
        var otherAfter;
    </script>
</body>
</html>

案例
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <script>
        //案例1
        var num=10;
        fn();
        function fn() {
            console.log(num); //由于变量提升原因,输出undefined
            var num=20;
        }
        // 相当于
        // var num;
        // function fn() {
        //     var num;
        //     console.log(num);
        //     num=20;
        // }
        // num=10
        // fn();
        //案例4
        f1();
        console.log(c);
        console.log(b);
        console.log(a);
        function f1() {
            var a=b=c=9;
            console.log(a);
            console.log(b);
            console.log(c);
        }
        // 相当于
        // b=9; c=9;
        // function f1() {
        //     // var a=b=c=9; 
        //     //相当于var a=9; b=9; c=9;并不是集体赋值,b和c没有var,是全局作用域
        //     //集体赋值为 var a=9,b=9,c=9;
        //     var a;
        //     a = 9;
        //     console.log(a); //a=9
        //     console.log(b); //b=9
        //     console.log(c); //c=9
        // }
        // f1();
        // console.log(c); //9
        // console.log(b); //9
        // console.log(a); //报错：Uncaught ReferenceError: a is not defined
    </script>
</body>
</html>

# Proxy

In [13]:
%%js

const target = {
	msg1: 'hello',
	msg2: 'everyone'
}

const handler1={}
const proxy1=new Proxy(target, handler1)
console.log(proxy1.msg1, proxy1.msg2) // hello everyone

const handler2={
	get(target,prop, receiver) {
		return 'world'
	}
}
const proxy2=new Proxy(target, handler2)
console.log(proxy2.msg1, proxy2.msg2) // world world

const handler3={
	get(target,prop,receiver) {
		if (prop === 'msg2') {
			return 'world'
		}

		return Reflect.get(...arguments)
	}
}
const proxy3=new Proxy(target, handler3)
console.log(proxy3.msg1, proxy3.msg2)


<IPython.core.display.Javascript object>

# Event

## 事件注册
```javascript
eventTarget.addEventListerner(type, listener[, useCapture])
// type: 事件类型, click、mouseover、不带on
// listerner: 事件处理函数
// useCapture: 可选,true/false, 默认值false
//     - true 表示在事件捕获阶段调用事件处理程序
//     - false 表示在事件冒泡阶段调用事件处理程序
```

## 事件删除
```javascript
// 传统方式
eventTarget.onclick = null;

// 方法监听方式
eventTarget.removeEventListerner(type, listerner[, useCapture]);
eventTarget.detachEvent(eventNameWithOn, callback);
```

## DOM事件流
- 事件流描述的是从页面接收事件的顺序
- 事件发生时会在元素节点之间按照特定的顺序传播
- 分为3个阶段:
    - 捕获阶段
    - 当前目标阶段
    - 冒泡阶段
    - 捕获阶段(document->html->body->div) => 目标阶段 => 冒泡阶段(div->body->html->document)
- JavaScript代码只能执行捕获或者冒泡其中的一个阶段
- 实际开发中很少使用事件捕获，更关注事件冒泡
- onclick/attachEvent只能得到冒泡阶段
- 有些事件没有冒泡,如onblur、onfocus、onmouseenter、onmouseleave

## 事件对象
```javascript
eventTarget.onclick = function(event) {}
eventTarget.addEventListener('click', function(event){})
// 这里的event就是事件对象,经常写成e或者evt
// event对象代表事件的状态，如键盘按键状态、鼠标位置等
```
- 常见属性和方法
    - e.target 返回触发事件的对象
    - e.type 事件类型
    - e.preventDefault() 阻止默认事件（默认行为）标准，如不让链接跳转
    - e.stopPropagation() 阻止冒泡 标准

## 常用事件

### 常用鼠标事件
- onclick
- onmouseover 鼠标经过
- onmouseout 鼠标离开
- onfoucs 获得鼠标焦点
- onblur 失去鼠标焦点
- onmousemove 鼠标移动
- onmouseup 鼠标弹起
- onmousedown 鼠标按下
- contextmenu
    ```javascript
    // 禁止鼠标右键菜单
    document.addEventListener('contextmenu', function(e) {
        e.preventDefault()
    })
    ```
- selectstart
    ```javascript
    // 禁止鼠标选中
    document.addEventListener('selectstart', function(e) {
        e.preventDefault()
    })
    ```
- MouseEvent
    ```javascript
    e.clientX 鼠标相对于浏览器窗口可视区的x坐标
    e.clientY 鼠标相对于浏览器窗口可视区的y坐标
    e.screenX 鼠标相对于电脑屏幕x坐标
    e.screenY 鼠标相对于电脑屏幕y坐标
    ```

### 常用键盘事件
- onkeydown 识别功能键，不区分大小写
- onkeypress 不识别功能键，区分大小写
- onkeyup 识别功能键，不区分大小写
- KeyboardEvent
    ```javascript
    e.keyCode
    ```

# BOM
- window
    - document
    - location
    - navigation
    - screen
    - history

## 事件

### load事件
- 当文档内容完成加载完成会触发该事件(包括图像、脚本、css等),调用处理函数

    ```javascript
    // 传统方式: 只能写一次，多次以最后一次为准
    window.onload = function() {}

    // addEventListener方式没有次数限制
    window.addEventListener('load', function(){});
    ```

### DOMContentLoaded事件
- 仅当DOM加载完成是触发，不包含css,图片等
- 页面图片很多时使用onload需要比较长的时间，此时使用DOMContentLoaded比较合适

### resize事件
- 调整窗口大小事件
- window.innerWidth当前屏幕的宽度

## 定时器
- window.setTimeout(调用函数, [延迟的毫秒数]);
- window.setInterval(回调函数, [间隔的毫秒数]);
- window.clearInterval(intervalID); 取消先前通过setInterval()建立的定时器

## this
- 一般情况下this的最终指向的是那个调用它的地方
- 全局作用域或者普通函数中this执行window

## location对象
- 属性
    - location.href 整个URL
    - location.host
    - location.port
    - location.pathname 路径
    - location.search 参数
    - location.hash `#`后面的内容，常见于链接锚点
- 方法
    - location.assign() 跟href一样,可跳转页面(也称为重定向页面)
    - location.replace() 替换当前页面,不记录历史,不能后退
    - location.reload() 重新加载页面

## navigator对象
- 包含有关浏览器的信息,最常用的是userAgent

## history对象
- 包含用户访问过的URL
- back() 后退
- forward() 前进
- go(参数) 前进或后退,go(1)前进一个页面,go(-1)后退一个页面